# `dsolve` examples

The objective of `dsolve` is to be easy to use. The system requires a list of strings written in $\LaTeX$, a dictionary of parameters with numerical values used in the calibration, and the specification of the state variables `x`, the forward-looking variables `p`, and the shocks `z`. 

Once the system is initialized and solved, `simulate_path` computes the path of endogenous variables given a sequence of unpredicted shocks `z` and a initial state `x0`.

In [1]:
sys.path.append('/bbkinghome/mbarrera/git_supply/dsolve/src')

In [1]:
import dsolve

In [2]:
from dsolve import DynamicSystem
from dsolve.atoms import Variable, Parameter
import numpy as np
import matplotlib.pyplot as plt

### Variables in `dsolve`

A variable in `dsolve` is any symbol that depends on `t`, with the possibility of having an expected value operator. There are many ways to flexibly reference to the same variable as the following example shows.

In [3]:
print(Variable('E_{t}[\pi^{w}_{t+1}]'))
print(Variable('E\pi^w_{t+1}'))
print(Variable('E_t\pi_{t+1}^w'))
print(Variable('E[\pi_{t+1}^w]'))
Variable('E_{t}[\pi^{w}_{t+1}]').sympy

E_{t}[\pi^{w}_{t+1}]
E_{t}[\pi^{w}_{t+1}]
E_{t}[\pi^{w}_{t+1}]
E_{t}[\pi^{w}_{t+1}]


E_{t}[\pi^{w}_{t+1}]

### AR(1)

The simples example is an $AR(1)$ process. Since it has no forward-looking variables, `p` does not need to be specified. 
$$x_t=\rho x_{t-1}+\sigma\epsilon_t$$
$x_{-1}=0, \epsilon_0=1, \epsilon_t=0\ \text{for}\ t>0$

In [4]:
eq = ['x_{t}=\rho*x_{t-1}+\sigma*eps_{t}']  # define the law of motion of x_t

calibration = {'\rho':0.8,'\sigma':1}      # give numerical values to parameters.

system = DynamicSystem(eq, x='x_{t-1}', z='eps_{t}', calibration=calibration)
impulse_response = dsolve.simulate(system, z = {'eps_{0}':1}, T=12)
impulse_response
#system.simulate(x0 = 0, z = {'eps_{0}':1}, T=12)
#mit_shock = system.simulate(x0 = 0, z = {'eps_{0}':1}, T=12)
#mit_shock = system.simulate(x0 = 0, z = {'eps_{t}':np.array([1.,0,0,0,0,0,0,0,0,0,0])})

#mit_shock.plot('x_{t-1}')

,t,x_{t-1},x_{t},eps_{t}
0,0,0.000000,1.000000,1.0
1,1,1.000000,0.800000,0.0
2,2,0.800000,0.640000,0.0
3,3,0.640000,0.512000,0.0
4,4,0.512000,0.409600,0.0
5,5,0.409600,0.327680,0.0
6,6,0.327680,0.262144,0.0
7,7,0.262144,0.209715,0.0
8,8,0.209715,0.167772,0.0
9,9,0.167772,0.134218,0.0


### 2 equations NK model

$$\pi_t=\beta E[\pi_{t+1}]+\kappa y_{t}+u_t$$
$$y_t=E[y_{t+1}]+(1-\phi)E[\pi_{t+1}]+u_t$$

In [5]:
eq=[
    '\pi_{t}=\beta*E\pi_{t+1}+\kappa*y_{t}+u_{t}',
    'y_{t}=Ey_{t+1}+(1-\phi)*E[\pi_{t+1}]+\epsilon_{t}',
    '\epsilon_{t} = \rho_v*\epsilon_{t-1}+v_{t}'
]
calibration = {'\beta':0.98,'\kappa':0.1,'\phi':1.1,'\rho_v':0.8}
system = DynamicSystem(eq, x='\epsilon_{t-1}', p='\pi_{t},y_{t}', z='v_{t},u_{t}', calibration=calibration)

dsolve.simulate(system, z = {'v_{0}':1}, T=12)

#mit_shock = system.simulate(x0=0, z = {'v_{0}':1}, T=12)

#fig,ax = plt.subplots()
#system.plot(ax,mit_shock,vars='\pi_t')
#ax.legend()
#fig.show()


,t,\epsilon_{t-1},\epsilon_{t},\pi_{t},y_{t},v_{t},u_{t}
0,0,0.000000,1.000000,1.953125,4.218750,1.0,0.0
1,1,1.000000,0.800000,1.562500,3.375000,0.0,0.0
2,2,0.800000,0.640000,1.250000,2.700000,0.0,0.0
3,3,0.640000,0.512000,1.000000,2.160000,0.0,0.0
4,4,0.512000,0.409600,0.800000,1.728000,0.0,0.0
5,5,0.409600,0.327680,0.640000,1.382400,0.0,0.0
6,6,0.327680,0.262144,0.512000,1.105920,0.0,0.0
7,7,0.262144,0.209715,0.409600,0.884736,0.0,0.0
8,8,0.209715,0.167772,0.327680,0.707789,0.0,0.0
9,9,0.167772,0.134218,0.262144,0.566231,0.0,0.0


### Test with static equations

`dsolve` also handles static equations. In order to reduce the size of the system, those can be specified using the `s` argument. Equations that define static variables must be written with the `s` variable in the left hand side.

Prior to solving the system, `dsolve` will substitute the variables in `s` into the other equations. If `s` is not specified and the system contains static variables, or is not easy to express them, `dsolve` will also solve the system but the matrices will be bigger. 

In [6]:
eq=[
    '\pi_{t}=b*E\pi_{t+1}+kappa*y_{t}',
    'y_{t}=Ey_{t+1}-(i_t-E\pi_{t+1}+rho)',
    'v_t = rho_v*v_{t-1}+eps_t',
    'i_t=rho+phi*\pi_{t}+v_t'  # static equation, written in the form i_t=.... and specified by s='i_t'
]
calibration = {'b':0.98,'kappa':0.1,'phi':1.1,'rho_v':0.8, 'rho':0.02}
system = DynamicSystem(eq, x='v_{t-1}', p='\pi_t, y_t', z='eps_t', s='i_t', calibration=calibration)
dsolve.simulate(system, z = {'eps_{0}':1}, T=12)
# mit_shock = system.simulate(x0=0, z = {'eps_{0}':0.25},T=12)
# fig,ax = plt.subplots(ncols=2,figsize=(10,3))
# system.plot(ax[0], mit_shock, vars='\pi_t, y_t')
# ax[0].legend()
# system.plot(ax[1], mit_shock, vars='i_t')
# system.plot_expr(ax[1], mit_shock, expr='i_t-E_t[\pi_{t+1}]')
# ax[1].legend()
# fig.show()

,t,v_{t-1},v_{t},\pi_{t},y_{t},eps_{t},i_{t}
0,0,0.000000,1.000000,1.366120,2.950820,1.0,2.52273224043716
1,1,1.000000,0.800000,1.092896,2.360656,0.0,2.02218579234973
2,2,0.800000,0.640000,0.874317,1.888525,0.0,1.62174863387978
3,3,0.640000,0.512000,0.699454,1.510820,0.0,1.30139890710383
4,4,0.512000,0.409600,0.559563,1.208656,0.0,1.04511912568306
5,5,0.409600,0.327680,0.447650,0.966925,0.0,0.840095300546450
6,6,0.327680,0.262144,0.358120,0.773540,0.0,0.676076240437160
7,7,0.262144,0.209715,0.286496,0.618832,0.0,0.544860992349728
8,8,0.209715,0.167772,0.229197,0.495065,0.0,0.439888793879783
9,9,0.167772,0.134218,0.183358,0.396052,0.0,0.355911035103826


### Constants

In [7]:
eq = ['x_{t}=mu+rho*x_{t-1}+sigma*eps_{t}']  # define the law of motion of x_t

calibration = {'rho':0.8,'sigma':1,'mu':1}      # give numerical values to parameters.

system = DynamicSystem(eq, x='x_{t-1}', z='eps_{t}', calibration=calibration)
dsolve.simulate(system, x0 = 0, z = {'eps_{0}':1}, T=12)

,t,x_{t-1},x_{t},eps_{t}
0,0,0.000000,1.000000,1.0
1,1,1.000000,0.800000,0.0
2,2,0.800000,0.640000,0.0
3,3,0.640000,0.512000,0.0
4,4,0.512000,0.409600,0.0
5,5,0.409600,0.327680,0.0
6,6,0.327680,0.262144,0.0
7,7,0.262144,0.209715,0.0
8,8,0.209715,0.167772,0.0
9,9,0.167772,0.134218,0.0


### VAR

Shocks can also be specified by its full (unexpected) path realization.

\begin{align}
x_{t+1}&=ax_t+by_t+\epsilon_t\\
y_{t+1}&=cx_t+dy_t+\nu_t
\end{align}

In [8]:
eq=[
    'x_{t+1}=a*x_{t}+b*y_{t}+eps_{t}',
    'y_{t+1}=c*x_{t}+d*y_{t}+nu_{t}'
]
calibration = {'a':0.8,'b':0.5,'c':-0.7,'d':-0.2}
system = DynamicSystem(eq, x='x_t,y_t', z='eps_t, nu_t', calibration=calibration)
mit_shock = dsolve.simulate(system, 
                            x0=(0,0), 
                            z = {'eps_{t}':np.array([1,0,0,0,0,0,0,0,-0.25,0,0,0,0,0,0,0,0,0])})
#fig,ax = plt.subplots()
#system.plot(ax,mit_shock,vars='x_{t},y_{t}')
#ax.legend()
#fig.show()